In [1]:
import os
os.chdir('../')
os.getcwd()

'/home/who/Documents/Coding/MLP-2_Chicken-Disease-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,config_path = CONFIG_FILE_PATH,params_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_prep_callback_config(self)-> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])
        
        prep_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prep_callback_config
        

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

2025-01-03 15:03:51.964908: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-03 15:03:51.969394: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-03 15:03:51.988583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735896832.021140   18430 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735896832.029796   18430 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-03 15:03:52.056108: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [6]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d_%H:%M:%S')
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
        ]

In [7]:
try:
    config = ConfigurationManager()
    prep_cb_config = config.get_prep_callback_config()
    
    prep_cb = PrepareCallback(config=prep_cb_config)
    callback_list = prep_cb.get_tb_ckpt_callback()
except Exception as e:
    raise e

[2025-01-03 15:03:55,436: INFO: common: YAML file: config/config.yaml read successfully]
[2025-01-03 15:03:55,441: INFO: common: YAML file: params.yaml read successfully]
[2025-01-03 15:03:55,442: INFO: common: Created directory at: 'artifacts']
[2025-01-03 15:03:55,445: INFO: common: Created directory at: 'artifacts/prepare_callbacks/checkpoint_dir']
[2025-01-03 15:03:55,447: INFO: common: Created directory at: 'artifacts/prepare_callbacks/tensorboard_log_dir']
